# Lesson 1
## Backtesting simple strategy (easy way)

## Description

1. Asset: 
  - XAUUSD

2. Period: 
  - 2024/06/01 - 2025/01/01

3. Entry conditions:
  - Time Frame: 30min
  - buy at 10:30 AM UTC, if prev candle closed bullish.
  - sell at 12:00 AM UTC, if prev candle closed bearish.

4. Exit conditions:
  - exit after 1 hour.

5. Sizing: 
  - 1lot.

## 0. Imports

In [ ]:
import pandas as pd
import vectorbt as vbt

## 1. Data preparation

In [ ]:
data = pd.read_csv("data.csv", parse_dates=["Time"])
data

## 2. Entries / Exits 

In [ ]:
long_entries = pd.Series(False, index=data.index)
short_entries = pd.Series(False, index=data.index)
long_exits = pd.Series(False, index=data.index)
short_exits = pd.Series(False, index=data.index)

In [ ]:
is_bullish_opened = False
is_bearish_opened = False

for i, candle_data in enumerate(data.itertuples()):
  dt = candle_data.Time
  time_ = str(dt.time()).split(":")

  if is_bullish_opened == True:
    if float(time_[0]) == 11 and float(time_[1]) == 30:
      long_exits.iloc[i] = True
      is_bullish_opened = False

  if is_bearish_opened == True:
    if float(time_[0]) == 13 and float(time_[1]) == 0:
      short_exits.iloc[i] = True
      is_bearish_opened = False

  if float(time_[0]) == 10 and float(time_[1]) == 30: # for bullish entry
    prev_row = data.iloc[i-1]
    prev_row_open = float(prev_row["Open"].item())
    prev_row_close = float(prev_row["Close"].item())

    if prev_row_open < prev_row_close: # bullish
      long_entries.iloc[i] = True
      is_bullish_opened = True

  if float(time_[0]) == 12 and float(time_[1]) == 0: # for bearish entry
    prev_row = data.iloc[i-1]
    prev_row_open = float(prev_row["Open"].item())
    prev_row_close = float(prev_row["Close"].item())

    if prev_row_open > prev_row_close: # bearish
      short_entries.iloc[i] = True
      is_bearish_opened = True
      

In [ ]:
data[long_exits == True]

## 3. Backtest

In [ ]:

pf = vbt.Portfolio.from_signals(
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    price=data["Close"],
    entries=long_entries,
    exits=long_exits,
    short_entries=short_entries,
    short_exits=short_exits,
    size=1.0,
    fees=0.001,
    init_cash=10000,
    freq="1min"
)

print(pf.stats())
pf.plot()


In [ ]:
data.iloc[31]